# PrivateQ

## Set-up

Import necessary libraries and configurations

In [1]:
# Set up
import os
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")

from dotenv import load_dotenv; load_dotenv()

import ray

In [2]:
from privateq.config import ROOT_DIR

# Set up ray with credentials and start ray
ray.init(runtime_env={
    "env_vars": {
        "OPENAI_API_BASE": os.environ["OPENAI_API_BASE"],
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"], 
    },
    "working_dir": str(ROOT_DIR)
})

# Show resources
ray.cluster_resources()

2024-01-01 13:27:03,967	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-01-01 13:27:03,985	INFO packaging.py:530 -- Creating a file package for local directory '/Users/tiluan/git/privateq'.
2024-01-01 13:27:03,991	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_d6d5776466f04af5.zip' (0.04MiB) to Ray cluster...
2024-01-01 13:27:04,007	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_d6d5776466f04af5.zip'.


{'memory': 3153867572.0,
 'node:127.0.0.1': 1.0,
 'CPU': 8.0,
 'object_store_memory': 1576933785.0,
 'node:__internal_head__': 1.0}

## Data Preparation

Load, chunk, embed and index data

In [3]:
# Import model parameters
from privateq.config import EMBEDDING_DIMENSIONS, MAX_CONTENT_LENGTH

print(f'EMBEDDING_DIMENSIONS: {EMBEDDING_DIMENSIONS}\nMAX_CONTENT_LENGTH: {MAX_CONTENT_LENGTH}')

EMBEDDING_DIMENSIONS: 768
MAX_CONTENT_LENGTH: 8192


In [4]:
# Load Data using Ray

from pathlib import Path
from privateq.config import FILE_DIR

# Find documents
DOCS_DIR = Path(FILE_DIR, os.environ.get("DOCS_DIR"))
print(f'DOCS_DIR: {DOCS_DIR}')
assert DOCS_DIR.exists(), f'{DOCS_DIR} does not exist.'

# Filter out html files and load them as ray dataset
ds = ray.data.from_items([{"path": path} for path in DOCS_DIR.rglob("*.html") if not path.is_dir()])
print(f"{ds.count()} documents")

DOCS_DIR: /Users/tiluan/git/.tmp/docs.ray.io/en/master
1967 documents


In [5]:
"""
Extract content from dataset, identify sections in html pages 
and extract text between them. Then save all of them to a list
of dictionaries that map text of a section to a specific url 
with specific anchor id.
"""
import matplotlib.pyplot as plt
from privateq.data_process import extract_sections

sample_html_fp = Path(FILE_DIR, "docs.ray.io/en/master/rllib/rllib-env.html")
extract_sections({"path": sample_html_fp})[0]

ModuleNotFoundError: No module named 'bs4'

In [6]:
# Shutdown Ray to clean resources
ray.shutdown()